In [21]:
import numpy as np
import datetime
from urllib.request import urlopen
import time
import sys
import math

In [22]:
#IMPORTANT
#if you want to run this to a custom date change loc_date at the
#last block of code after the function assignment
#the program will run for the three months before loc_date

#finds difference in number of days between two dates
#helper function for month_days
def month_days(month,year):
    days={1:31,2:28,3:31,4:30,5:31,6:30,7:31,8:31,9:30,10:31,11:30,12:31}
    if year%4==0 or year%400==0 :
        days[2]+=1
    return days[month]
#month_days(2,2020)

In [23]:
#function to increment a date by a number of days
def day_inc(start_date,days):
    m_days_left=month_days(start_date[1],start_date[0])-start_date[2]
    if m_days_left<days:
        start_date[1]+=1
        start_date[2]=1
        days-=m_days_left+1
        if(start_date[1]>12):
            start_date[0]+=1
            start_date[1]=1
        return day_inc(start_date,days)
    else:
        start_date[2]+=days
        days=0
    #final date or end date is returned below
    return start_date
#function to decrement a date by a number of days
def day_dec(start_date,days):
    if(start_date[2]<=days):
        days-=start_date[2]
        if(start_date[1]>1):
            start_date[1]-=1
        else:
            start_date[1]=12
            start_date[0]-=1
        start_date[2]=month_days(start_date[1],start_date[0])
        return day_dec(start_date,days)
    else:
        start_date[2]-=days
    return start_date
#day_dec([2020,3,31],32)
'''
#timing the functions for testing
start=time.time()
for i in range(1000):
    day_inc([2020,3,31],1)
end=time.time()
print("Time taken inc",end-start)
start=time.time()
for i in range(1000):
    day_dec([2020,3,31],1)
end=time.time()
print("Time taken dec",end-start)
'''

'\n#timing the functions for testing\nstart=time.time()\nfor i in range(1000):\n    day_inc([2020,3,31],1)\nend=time.time()\nprint("Time taken inc",end-start)\nstart=time.time()\nfor i in range(1000):\n    day_dec([2020,3,31],1)\nend=time.time()\nprint("Time taken dec",end-start)\n'

In [24]:
#makes a list of hospitals to help with file handling
def hospital_list(n):
    arr=[]
    for i in range(n):
        arr.append("./Hospitals/hospital_"+str(i+1))
    return arr

In [25]:
def hosp_cap(hospital):
    file=open(hospital,'r')
    hos_cap=file.readline().split()
    hos_cap=int(hos_cap[-1])
    file.close()
    return hos_cap
def hospital_analysis(hospital):
    '''
    file=open(hospital,'r')
    hos_cap=file.readline().split()
    hos_cap=int(hos_cap[-1])
    file.close()
    print("Hospital capacity of: ",hospital,":- ",hos_cap)'''
    file_list = np.genfromtxt(hospital,delimiter=" ",dtype=( int,"|U12", int, int,"|U200"),comments='#')
    #pat mod list is a dictionary with dates and number of [admissions,releases] per day in a list as value for keys
    pat_mod_list={}
    current_occupancy=0
    dead_monthly=0
    healed_monthly=0
    prev_date=''
    curr_date=''
    i=-1
    start=0
    iter_date=[]
    for ele in file_list:
        curr_date=ele[1]
        if(start==0):
            iter_date=curr_date
            start=1
        if(curr_date!=prev_date):
            pat_mod_list[curr_date]=[1,0,0,0]
            prev_date=curr_date
            i+=1
        else:
            pat_mod_list[curr_date][0]+=1
        if(ele[3]==1):
            dead_monthly+=1
        elif(ele[3]==0):
            healed_monthly+=1
        elif(ele[3]==-1):
            current_occupancy+=1
    for ele in file_list:
        curr_date=ele[1]
        release_date="%d-%d-%d"%tuple(day_inc(list(map(int,curr_date.split("-"))),ele[2]))
        #adding people who are released or dead only to the releases
        if(ele[3]!=-1):
            if(ele[3]==1):
                #dead
                pat_mod_list[release_date][1]+=1
            else:
                #recovered
                pat_mod_list[release_date][2]+=1
    pat_mod_len=len(pat_mod_list)
    curr_pat_num=0
    for i in range(pat_mod_len):
        #updating with new admissions-dead-recovered
        curr_pat_num+=pat_mod_list[iter_date][0]-pat_mod_list[iter_date][1]-pat_mod_list[iter_date][2]
        pat_mod_list[iter_date][3]=curr_pat_num
        #incrementing date below
        iter_date="%d-%d-%d"%tuple(day_inc(list(map(int,iter_date.split("-"))),1))
    return pat_mod_list,dead_monthly,healed_monthly,current_occupancy

for i in range(1,11):        
    pat_admitted,dead_monthly,healed_monthly,current_occupancy=hospital_analysis("./Hospitals/hospital_%d.txt"%i)
    #print("Hospital %d"%i)
    #[print(key,value) for key, value in pat_admitted.items()]
    #line below was for a list, now modified to a dictionary with list comprehension to print above
    #print(*pat_admitted, sep= "\n")

In [26]:
#for fitting the exponential 
#the exponential is taken of the form y=a*e^(bx) where a and b are the fitting parameters
#Least square fit can be done on this taking log y=loga+bx
#x refers to number of days passed since first day of observation
#y gives the number of patients newly admitted or those who have died
#a is a related to initial number of patients
#b is the significant parameter here which we need to find. 
#finding b will let us find the rate at which the disease is spreading and evaluate how dangerous it is
#Here Matrix A is (1,x) which is used to find loga,b
def least_square(A,b):
    At=np.transpose(A)
    AtAi=np.linalg.inv(np.dot(At,A))
    return np.linalg.multi_dot([AtAi,At,b])

In [27]:
#fitting specific to our cases
#for y=a*m**x=a*e**(b*x)
def leastsq_xy(xarr,yarr):
    nyarr=[]
    for ele in yarr:
        if ele<0.01:
            nyarr.append(math.log(0.01))
        else:
            nyarr.append(math.log(ele))
    matA=np.column_stack(([1]*len(xarr),xarr))
    arrb=np.asarray(nyarr)
    soln=least_square(matA,arrb)
    return math.exp(soln[1])

In [28]:
#fitting algorithm has been taken from
#http://www.davdata.nl/math/expfitting.html
#please refer to the above page for the algorithm
def biasexpfitting(xarr,yarr):
    diffx = [] 
    cx=[]
    for x, y in zip(xarr[0::], xarr[1::]): 
        diffx.append(y-x) 
        cx.append((x+y)/2.0)
    #print("diffx",diffx)
    #print("cx",cx)
    diffy = [] 
    dq=[]
    i=0
    for x, y in zip(yarr[0::], yarr[1::]): 
        diffy.append(y-x) 
        dq.append(1.0*diffy[i]/diffx[i])
        i+=1
    #print("diffy",diffy)
    #print("dq",dq)
    b=[]
    for x1,x2,y1,y2 in zip(cx[0::],cx[1::],dq[0::],dq[1::]):
        if(y2*y1>0):
            b.append(math.log(y2/y1)/(x2-x1))
        else:
            b.append(1)
    #first approximation value
    approxb=sum(b)/len(b)
    a=[]
    #print("approxb",approxb)
    #print("xarr",xarr)
    for x1,x2,y in zip(xarr[0::],xarr[1::],diffy[0:]):
        denom=math.exp(approxb*x2)-math.exp(approxb*x1)
        #print(math.exp(approxb*x2),"and ",x2)
        #print(math.exp(approxb*x1),"and ",x1)
        if denom==0:
            #print("same x indices error")
            a.append(1)
        else:
            a.append(y/denom)
    approxa=sum(a)/len(a)
    c=[]
    for x,y in zip(xarr[0::],yarr[0::]):
        c.append(y-approxa*(math.exp(approxb*x)))
    #print("carr",c)
    approxc=sum(c)/len(c)
    ymod=[ele-approxc for ele in yarr]
    return leastsq_xy(xarr,ymod)

In [29]:
#Testing the fitting function
'''
noise=8
bias=10
yarr=[int(round(bias+math.pow(1.5,i)-noise)+np.random.randint(0,2*noise)) for i in range(7)]
print(yarr)
xarr=list(range(len(yarr)))
#print(xarr)
print(biasexpfitting(xarr,yarr))
'''

'\nnoise=8\nbias=10\nyarr=[int(round(bias+math.pow(1.5,i)-noise)+np.random.randint(0,2*noise)) for i in range(7)]\nprint(yarr)\nxarr=list(range(len(yarr)))\n#print(xarr)\nprint(biasexpfitting(xarr,yarr))\n'

In [30]:
def Fitting_param_month(hospital):
    pat_admitted,dead_monthly,healed_monthly,current_occupancy=hospital_analysis(hospital)
    #to create a copy
    admitted_curve=dict(pat_admitted)
    for key,value in admitted_curve.items():
        admitted_curve[key]=value[0]
    #to create a copy
    death_curve=dict(pat_admitted)
    for key,value in death_curve.items():
        death_curve[key]=value[1]
    occupancy_curve=dict(pat_admitted)
    for key,value in occupancy_curve.items():
        occupancy_curve[key]=value[3]
    #arr=list(death_curve.items())
    #print(arr[-1][0])
    #[print(key,value) for key, value in death_curve.items()]
    #[print(key,value) for key, value in admitted_curve.items()]
    return admitted_curve,death_curve,current_occupancy,occupancy_curve
#print(Fitting_param_month("hospital_2.txt"))

In [31]:
#function to get PC time and timezone time online
def comp_online_time():
    #getting computer date
    d=datetime.datetime.now()
    comp_date=[d.year,d.month,d.day]
    #print(comp_date)
    #getting online date below with exception handling for no internet
    try:
        res = urlopen('http://just-the-time.appspot.com/')
        result = res.read().strip()
        result_str = result.decode('utf-8')
        time=result_str.split()
        #UTC date is obtained from a google app service below which is then changed to Indian time
        days_date=list(map(int,time[0].split('-')))
        hour_time=list(map(int,time[1].split(':')))
        if(hour_time[0]>=19 or (hour_time[0]>=18 and hour_time[1]>=30)):
            days_date=day_inc(days_date,1)
    except:
        print("No Internet")
        print("Oops!",sys.exc_info()[0],"occured.")
        days_date=0
    return days_date,comp_date
#below function takes care of error rising from delay in calculating online and local date.
def current_date():
    flag=1
    online,local=comp_online_time()
    if(online!=local):
        online,local=comp_online_time()
        if(online!=local):
            flag=0 
    if not(online):
        flag=-1
    #returns current online synced date and whether local time is also matching 1 for matching 0 for mismatch
    return online,local,flag
current_date()

([2020, 4, 30], [2020, 4, 30], 1)

In [32]:
#online date is the actual date and loc_date is the current date. 
#Data is taken till local date
def exponential_param(hospital,file,loc_date):
    file.write("ID: %s\n"%hospital)
    cap_hospital=hosp_cap("./Hospitals/%s"%hospital)
    file.write("Cap: %d\n"%cap_hospital)
    
    #last date is excluded both to prevent errors and prevent incomplete data from coming in
    three_months=day_dec(loc_date[:],92)
    one_month=day_dec(loc_date[:],32)
    one_week=day_dec(loc_date[:],9)
    #print(three_months,one_month,one_week) 
    admitted_curve,death_curve,current_occupancy,occupancy_curve=Fitting_param_month("./Hospitals/%s"%hospital)
    thm_arr=[0]*90
    onm_arr=[0]*30
    onw_arr=[0]*7
    #print("Keys")
    #print(admitted_curve.keys())
    #getting log of no of patients in different periods for fitting and finding rise parameter
    #and Doing least square fitting for each range of days
    if("%d-%d-%d"%tuple(three_months) in occupancy_curve.keys()):
        for i in range(90):
            thm_arr[i]=occupancy_curve["%d-%d-%d"%tuple(three_months)]
            three_months=day_inc(three_months,1)
        #for 90 days
        xarr=range(90)
        thm_param=biasexpfitting(xarr,thm_arr)
        file.write("3_m: %f\n"%thm_param)
        #print("Three month",list(map(math.exp,thm_arr)))
    if("%d-%d-%d"%tuple(one_month) in occupancy_curve.keys()):
        for i in range(30):
            onm_arr[i]=occupancy_curve["%d-%d-%d"%tuple(one_month)]
            one_month=day_inc(one_month,1)
        #for 30 days
        xarr=range(30)
        onm_param=biasexpfitting(xarr,onm_arr)
        file.write("1_m: %f\n"%onm_param)
        #print("One month",list(map(math.exp,onm_arr)))
    if("%d-%d-%d"%tuple(one_week) in occupancy_curve.keys()):
        for i in range(7):
            onw_arr[i]=occupancy_curve["%d-%d-%d"%tuple(one_week)]
            one_week=day_inc(one_week,1)
        #print("oneweek",onw_arr)
        #for 7 days
        xarr=range(7)
        onw_param=biasexpfitting(xarr,onw_arr)
        #print("One week",onw_param)
        file.write("1_w: %f\n"%onw_param)
        #print("One week",list(map(math.exp,onw_arr)))
    #file.write("Curr_occ: %d\n"%current_occupancy)
    file.write("Curr_occ_percent: %d\n"%(100.0*current_occupancy/cap_hospital))

In [33]:
#Testing if the algorithm works and detects increases properly
'''
testarr=[6, 1, 3, 4, 14, 33, 72]
minval=min(testarr)
testarr=[ele-minval+1 for ele in testarr]
print(testarr)
onw_arr=[]
for i in range(7):
    onw_arr.append(math.log(testarr[i]))
iters=range(1,8)
matA=np.column_stack(([1]*len(iters),iters))
arrb=np.asarray(onw_arr)
soln=least_square(matA,arrb)
print(math.exp(soln[1]))
'''

'\ntestarr=[6, 1, 3, 4, 14, 33, 72]\nminval=min(testarr)\ntestarr=[ele-minval+1 for ele in testarr]\nprint(testarr)\nonw_arr=[]\nfor i in range(7):\n    onw_arr.append(math.log(testarr[i]))\niters=range(1,8)\nmatA=np.column_stack(([1]*len(iters),iters))\narrb=np.asarray(onw_arr)\nsoln=least_square(matA,arrb)\nprint(math.exp(soln[1]))\n'

In [34]:
#compiles date for the given number of hospitals
def compile_data(num_hosp):
    #online date is the actual date and loc_date is the current date. 
    #Data is taken till local date
    #online data is to verify integrity of data and prevent unexpected data loss
    #if you want to do this to a custom date change loc_date
    on_date,loc_date,flag=current_date()
    if (flag==0):
        print("Time is not synced with the Internet properly. Please do that")
    if(flag==-1):
        print("Not connected to Internet. Connection is required for syncing")
    #verifying local date commented below
    #print("loc",loc_date)
    main_file=open('Compiled.txt','w')
    for it in range(num_hosp):
        exponential_param("hospital_%d.txt"%(it+1),main_file,loc_date)
    main_file.close()
    print("Done")

In [36]:
compile_data(62)
#here the number must match the number of entities being simulated 
#as set in previous file to make sure data is accurate

Done
